In [25]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [27]:
cie_10 = pd.read_excel("../data/external/CIE-10 - sin_puntos_y_X.xlsx")
regiones = (
    pd.read_excel("../data/external/Esquema_Registro-2023.xlsx", sheet_name=2, header=6)
    .dropna(how="all", axis=1)
    .iloc[:-1, :-1]
)
regiones["Código Comuna"] = regiones["Código Comuna"].astype(int)
regiones["Código Región"] = regiones["Código Región"].astype(int)

## 1. Análisis Sociodemografico a lo largo del tiempo

En este analisis se quieren responder las siguientes preguntas:

1. ¿Cuántas consultas hay por año?
2. ¿Cuáles son los diagnósticos más frecuentes por año?
3. ¿Cuál es el **rango etario** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnóstico?
4. ¿Cuál es el **género/sexo** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?
5. ¿Cuál es la **previsión** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?
6. ¿Cuál es el **servicio de salud** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?
7. ¿Cuál es la **comuna** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?

Para responder estas preguntas se utilizará la base de datos de consultas ambulatorias
del sistema HIS del Instituto Nacional del Tórax. La base de datos contiene consultas desde
el año 2016 al 2022. Esta base contiene los datos para saber la frecuencia por diagnóstico,
rango etario y género.

A pesar de lo anterior, la base de datos de consultas carece de los últimos 3 datos a analizar
(previsón, servicio de salud y comuna de residencia). Debido a esta razón, se utilizará la base
de TrackCare para obtener tales datos.

Además, se quiere responder cada una de estas preguntas para cada una de las especialidades de
las consultas, diagnósticos y por tipo de consulta.


In [28]:
df_track = pd.read_csv("../data/processed/datos_limpios_track.csv", sep=";", encoding="latin-1")
df_his = pd.read_csv(

    "../data/processed/datos_limpios_diagnosticos.csv", sep=";", encoding="latin-1"

)

In [29]:
df_track["hora_completa_cita"] = pd.to_datetime(df_track["hora_completa_cita"])
df_his["fecha_reserva"] = pd.to_datetime(df_his["fecha_reserva"])

In [30]:
print(
    f"La base de Track tiene citas agendadas desde {df_track['hora_completa_cita'].min()} "
    f"a {df_track['hora_completa_cita'].max()}"
)

print(
    f"La base de HIS tiene citas agendadas desde {df_his['fecha_reserva'].min()} a "
    f"{df_his['fecha_reserva'].max()}"
)
print()
print(f"La base de Track tiene {df_track.shape[0]} citas")
print(f"La base de HIS tiene {df_his.shape[0]} citas")

La base de Track tiene citas agendadas desde 2016-01-04 08:00:00 a 2022-12-30 20:30:00
La base de HIS tiene citas agendadas desde 2016-01-04 08:00:00 a 2022-12-30 14:50:00

La base de Track tiene 758350 citas
La base de HIS tiene 265441 citas


In [31]:
datos_pacientes_unicos_por_sesion = df_track.groupby(["id_paciente", "hora_completa_cita"]).head(1)

In [32]:
df_his_y_track = pd.merge(
    df_his,
    datos_pacientes_unicos_por_sesion,
    how="left",
    left_on=["id_paciente", "fecha_reserva"],
    right_on=["id_paciente", "hora_completa_cita"],
)

sin_datos_en_track = df_his_y_track[df_his_y_track.estadocita.isna()]

In [33]:
print(
    f"Luego de unir la base de HIS con Track, la primera ha quedado con "
    f"{df_his_y_track.shape[0]} consultas (de {df_his.shape[0]}). "
    f"Hay {sin_datos_en_track.shape[0]} consultas de HIS sin información en Track"
)

Luego de unir la base de HIS con Track, la primera ha quedado con 265441 consultas (de 265441). Hay 189 consultas de HIS sin información en Track


In [34]:
for paciente in sin_datos_en_track.id_paciente.unique():
    if paciente in df_track["id_paciente"]:
        print(f"{paciente} si esta")

Los resultados anteriores indican que ninguno de los pacientes faltantes se encuentra en la base
de Track (Y no ocurre debido a que los pacientes existen en la base, pero tienen una fecha de atencion distinta a la de HIS). 
Debido a lo anterior, estos pacientes carecen de datos sociodemográficos.

Una vez realizado la unión de las bases de datos es posible realizar el conteo anidado por
las variables sociodemográficas.

In [35]:
VARIABLES_SOCIODEMOGRAFICAS_INTERES = [
    "ano",
    "sexo_x",
    "comuna",
    "provincia",
    "region",
    "plan",
    "prevision",
    "tipoatencion",
    "rango_etario",
]

VARIABLE_A_CONTAR = ["nombre_diagnostico", "codigo_diagnostico"]

In [36]:
resultado_sociodemografico = (
    df_his_y_track.groupby(VARIABLES_SOCIODEMOGRAFICAS_INTERES)[VARIABLE_A_CONTAR]
    .value_counts()
    .reset_index(name="cantidad_de_consultas")
)

resultado_sociodemografico["comuna"] = resultado_sociodemografico["comuna"].str.upper()
resultado_sociodemografico = pd.merge(
    resultado_sociodemografico, regiones, how="left", left_on="comuna", right_on="Nombre Comuna"
).merge(cie_10, how="left", left_on="codigo_diagnostico", right_on="Código")

resultado_sociodemografico = resultado_sociodemografico.rename(
    columns={"Código Región": "codregion", "Código Comuna": "cod_comuna"}
)

resultado_sociodemografico["codregion"] = resultado_sociodemografico["codregion"].astype("Int32")
resultado_sociodemografico["cod_comuna"] = resultado_sociodemografico["cod_comuna"].astype("Int32")

In [37]:
resultado_sociodemografico.to_csv(
    "../data/interim/sociodemografico/desglose_sociodemografico.csv",
    index=False,
    encoding="latin-1",
    sep=";",
)

# Obtención de casos de Fibrosis Quística

En el Proyecto EPH del INT existe interés por saber la cantidad de consultas ambulatorias para la fibrosis quística. Por lo tanto, es necesario responder tal pregunta.

En primer lugar, hay que identificar todos los códigos diagnósticos que correspondan a la fibrosis quística.

In [38]:
# Genera una mask con las consultas que tengan 'fibrosis qu'. El nombre es pasado a minuscula
contienen_fibrosis_quistica = (
    resultado_sociodemografico["nombre_diagnostico"].str.lower().str.contains("fibrosis qu")
)

# Obtiene los codigos correspondientes a la fibrosis
resumen_fibrosis = resultado_sociodemografico[contienen_fibrosis_quistica]
codigos_fibrosis = resumen_fibrosis["codigo_diagnostico"].unique()
nombres_fibrosis = resumen_fibrosis["nombre_diagnostico"].unique()

print(
    f"Los códigos utiliados para la fibrosis quística son: {codigos_fibrosis} - {nombres_fibrosis}"
)

# Obtiene tabla dinámica con resultados de fibrosis
tabla_resumen_consultas_fibrosis = pd.pivot_table(
    resumen_fibrosis,
    index="rango_etario",
    columns="ano",
    values="cantidad_de_consultas",
    aggfunc="sum",
    fill_value=0,
    margins=True,
)

Los códigos utiliados para la fibrosis quística son: ['E840' 'E849' 'E841' 'E848' 'E84X' '23' '80'] - ['Fibrosis quística con manifestaciones pulmonares'
 'Fibrosis quística, sin otra especificación '
 'Fibrosis quística con manifestaciones intestinales'
 'Fibrosis quística con otras manifestaciones' 'Fibrosis quística '
 'Fibrosis quística infectada' 'Fibrosis Quistica sospecha']


In [39]:
tabla_resumen_consultas_fibrosis.to_excel("../data/interim/resumen_fibrosis_quistica_ambulatorio.xlsx")

In [40]:
# Define diagnosticos para los problemas de salud de interes
PROBLEMAS_DE_SALUD = {
    "FIBROSIS_QUISTICA": ["E840", "E841", "E849", "E848", "E84X", "23", "80"],  # Listo
    "HIPERTENSION_PULMONAR": ["I270", "I272", "64", "63", "65"],  # Listo
    "FIBROSIS_PULMONAR": ["J841"], # Por ver
    "ESCLERODERMIA": ["M340", "M348", "M349", "M34X"],  # Listo
    "MARFAN": ["Q874"],  # Listo
    "EXCAVATUM": ["Q676"],  # Listo
    "CARINATUM": ["Q677"],  # Listo
    "SILICOSIS": [], # Por ver
    "CARDIOPATIA_CONGENITA_ADULTO": [
        "Q201",
        "Q202",
        "Q203",
        "Q204",
        "Q205",
        "Q206",
        "Q208",
        "Q209",
        "Q210",
        "Q211",
        "Q212",
        "Q213",
        "Q214",
        "Q218",
        "Q220",
        "Q221",
        "Q222",
        "Q223",
        "Q224",
        "Q225",
        "Q228",
        "Q230",
        "Q231",
        "Q233",
        "Q240",
        "Q241",
        "Q244",
        "Q245",
        "Q246",
        "Q248",
        "Q249",
        "Q250",
        "Q251",
        "Q253",
        "Q254",
        "Q255",
        "Q256",
        "Q257",
        "Q258",
        "Q259",
    ],  # Listo
    "TRASPLANTE_DE_PULMON": [], # Proced
    "ECMO": [], # Proced
    "CANCER_DE_PULMON": ["C340", "C341", "C342", "C343", "C348", "C349", "C34X"], # Listo
    "CIRUGIA_CANCER_DE_PULMON": [], # Proced
    "TRASPLANTE_DE_CORAZON": [], # Proced
    "ENFERMEDAD_CORONARIA": [],  # Por Ver
    "ANEURISMA_DE_LA_AORTA": ["I711", "I712", "I713", "I715"],  # Listo
    "DISECCION_DE_LA_AORTA": [],  # Por Ver
    "MARCAPASOS": [], # Proced
    "INSUFICIENCIA_MITRAL": ["I051", "I340", "91-26"], # Listo
    "INSUFICIENCIA_AORTICA": ["I061", "I351", "91-24"], # Listo
    "APNEA_DEL_SUENO": ["G473"], # Listo
}

In [41]:
# FIltra solamente el periodo de interes
df_2019 = df_his_y_track.query("fecha_atencion < '2020-01-01'").copy()
# Limpia el nombre del diagnostico para buscar diversos diags para las patologias
df_2019["nombre_diagnostico"] = df_2019["nombre_diagnostico"].str.strip().str.lower()

In [42]:
resultados = {}
for problema_de_salud, codigos_diagnostico in PROBLEMAS_DE_SALUD.items():
    # Filtra el df del torax solamente a los diagnosticos del problema
    df_diagnostico = df_2019[df_2019["codigo_diagnostico"].isin(codigos_diagnostico)]

    # Calcula metricas totales para el periodo analizado
    consultas_totales = df_diagnostico.shape[0]
    pacientes_totales = len(df_diagnostico["id_paciente"].unique())

    # Calcula metricas desagregadas por anios
    resumen_diagnostico_agrupado = (
        df_diagnostico.groupby(["ano"])
        .agg(n_egresos=("codigo_diagnostico", "count"), n_pacientes=("id_paciente", "nunique"))
        .mean()
    )
    # Selecciona la primera fila y la segunda columna del resumen ("n_egresos")
    consultas_promedios = resumen_diagnostico_agrupado["n_egresos"]
    pacientes_promedio = resumen_diagnostico_agrupado["n_pacientes"]

    # Genera el string resumen de metricas totales y promedio
    linea_resumen_consultas = f"{consultas_totales} ({consultas_promedios:.2f})"
    linea_resumen_pacientes = f"{pacientes_totales} ({pacientes_promedio:.2f})"

    # Guarda los strings resumen en un DataFrame
    resumen_problema_salud = pd.DataFrame(
        {
            "consultas_totales_2016_a_2019 (suma - promedio)": [linea_resumen_consultas],
            "personas_atendidas_2016_a_2019 (suma - promedio)": [linea_resumen_pacientes],
        },
        index=[problema_de_salud],
    )

    resultados[problema_de_salud] = resumen_problema_salud

In [43]:
resumen_total_diags = pd.concat(resultados.values())
display(resumen_total_diags)

,consultas_totales_2016_a_2019 (suma - promedio),personas_atendidas_2016_a_2019 (suma - promedio)
FIBROSIS_QUISTICA,2024 (506.00),137 (87.50)
HIPERTENSION_PULMONAR,1736 (434.00),396 (215.25)
FIBROSIS_PULMONAR,7074 (1768.50),1613 (739.75)
ESCLERODERMIA,1 (1.00),1 (1.00)
MARFAN,28 (7.00),22 (5.75)
EXCAVATUM,248 (62.00),103 (42.50)
CARINATUM,41 (13.67),19 (8.33)
SILICOSIS,1770 (442.50),557 (234.50)
CARDIOPATIA_CONGENITA_ADULTO,0 (nan),0 (nan)
TRASPLANTE_DE_PULMON,1401 (350.25),153 (74.25)


In [44]:
resumen_total_diags.to_excel("../data/interim/resumen_area_influencia_ambulatorio.xlsx")

## Cómo identificar diagnósticos para problema de salud

Para identificar los diagnósticos para cada uno de los problemas de salud se utiliza la glosa del diagnósticos y se seleccionan los códigos afines. 

- A modo de ejemplo, se buscará los distintos códigos asociados al "Aneurisma".

In [71]:
GLOSA_A_BUSCAR = "insuficiencia reu"

codigos_asociados = (
    df_2019[df_2019["nombre_diagnostico"].str.contains(GLOSA_A_BUSCAR)]
    .groupby("codigo_diagnostico")["nombre_diagnostico"]
    .value_counts()
)

display(codigos_asociados)

codigo_diagnostico  nombre_diagnostico                                                       
I061                insuficiencia reumática de la válvula aórtica: regurgitación incompetente    141
I062                estenosis e insuficiencia reumaticas de la válvula aórtica                   167
Name: count, dtype: int64

In [46]:
# Modifica rangos etarios
df_his_y_track["rango_etario"] = pd.cut(
    df_his_y_track["edad"], [-np.inf, 15, 20, 45, 65, 80, np.inf], right=False
)

# Genera calculo de consultas y pacientes
resumen_fibrosis = (
    df_his_y_track[
        df_his_y_track["codigo_diagnostico"].isin(PROBLEMAS_DE_SALUD["FIBROSIS_QUISTICA"])
    ]
    .query("ano >= 2018 and ano <= 2021")
    .groupby(["ano", "rango_etario"])
    .agg(n_consultas=("codigo_diagnostico", "count"), n_pacientes=("id_paciente", "nunique"))
).reset_index()

In [47]:
# Pivota la tabla para formato de informe
tabla_dinamica_resumen_fibrosis = pd.pivot_table(
    resumen_fibrosis,
    index="rango_etario",
    columns="ano",
    values=["n_consultas", "n_pacientes"],
    aggfunc="sum",
    margins=True
)

# Une n_consultas y pacientes en una misma tabla con formato
resumen_string_fibrosis_n_consultas_pacientes = (
    tabla_dinamica_resumen_fibrosis["n_consultas"].astype(str)
    + " ("
    + tabla_dinamica_resumen_fibrosis["n_pacientes"].astype(str)
    + ")"
)

display(resumen_string_fibrosis_n_consultas_pacientes)

ano,2018,2019,2020,2021,All
rango_etario,,,,,
"[-inf, 15.0)",0 (0),0 (0),0 (0),0 (0),0 (0)
"[15.0, 20.0)",3 (1),1 (1),5 (2),39 (10),48 (14)
"[20.0, 45.0)",444 (82),530 (81),483 (84),570 (80),2027 (327)
"[45.0, 65.0)",46 (12),52 (11),66 (10),76 (10),240 (43)
"[65.0, 80.0)",3 (3),0 (0),1 (1),2 (2),6 (6)
"[80.0, inf)",1 (1),0 (0),0 (0),0 (0),1 (1)
All,497 (99),583 (93),555 (97),687 (102),2322 (391)


In [48]:
resumen_string_fibrosis_n_consultas_pacientes.to_excel(
    "../data/interim/resumen_fibrosis_quistica_ambulatorio.xlsx"
)